In [1]:
import pandas as pd 
import yfinance as yf 
from datetime import datetime, timedelta
from fredapi import Fred

In [2]:
fred_key = '3932ed65675a268ca1a4e0e68b10fa9b'
fred = Fred(api_key=fred_key)

In [3]:
# Setting the start and end date 
end_date = datetime.today()
start_date = end_date - timedelta(days=500*365)
print(start_date)

1525-01-20 12:43:20.831377


In [4]:
oil_api_keys = {
    "libya":"LBYNGDPMOMBD",
    "kazak":"KAZNGDPMOMBD",
    "qatar":"QATNGDPMOMBD",
    "iran":"IRNNGDPMOMBD",
    "kuwait":"KWTNGDPMOMBD",
    "uae":"ARENGDPMOMBD",
    "saudi":"SAUNGDPMOMBD",
    "iraq":"IRQNGDPMOMBD"
}

In [5]:
financial_api_keys = {
    "crude_price":"CL=F",
    "sp_500":"ES=F",
    "vix":"^VIX",
    "usd":"DX=F",
}

In [6]:
def get_oil_data(key):
    dataframes = []
    for i in key: 
        df = pd.DataFrame({f"{i}_oil_production":fred.get_series(f'{key[i]}')})
        dataframes.append(df)

    return dataframes

In [7]:
def get_financial_data(key,start,end):
    dataframes = []
    for i in key: 
        df = pd.DataFrame({f"{i}":yf.download(f"{key[i]}", start=start, end=end)['Adj Close']})
        dataframes.append(df)

    return dataframes

In [8]:
oil_df = get_oil_data(oil_api_keys)
oil_df

[            libya_oil_production
 2000-01-01          1.469428e+06
 2001-01-01          1.427842e+06
 2002-01-01          1.383184e+06
 2003-01-01          1.805719e+06
 2004-01-01          1.582475e+06
 2005-01-01          1.721293e+06
 2006-01-01          1.809577e+06
 2007-01-01          1.844703e+06
 2008-01-01          1.781000e+06
 2009-01-01          1.623000e+06
 2010-01-01          1.688000e+06
 2011-01-01          4.600000e+05
 2012-01-01          1.390000e+06
 2013-01-01          9.000000e+05
 2014-01-01          4.602740e+05
 2015-01-01          4.000000e+05
 2016-01-01          3.901644e+05
 2017-01-01          8.100000e+05
 2018-01-01          9.673633e+05
 2019-01-01          1.100000e+06
 2020-01-01          3.000000e+05
 2021-01-01          1.206667e+06
 2022-01-01          1.001250e+06
 2023-01-01          1.179167e+06
 2024-01-01          1.300000e+06
 2025-01-01          1.400000e+06,
             kazak_oil_production
 2000-01-01          7.270479e+05
 2001-01-01  

In [9]:
usa_oil_production = pd.read_csv('../data/usa_crude_oil_production.csv')
usa_oil_production = usa_oil_production.sort_values(['Month'],ascending=True)
usa_oil_production.rename(columns={'U.S. Field Production of Crude Oil Thousand Barrels per Day':'usa_oil_production'},inplace=True)
usa_oil_production['Month'] = pd.to_datetime(usa_oil_production['Month'], format='%b %Y').dt.strftime('%Y-%m-%d')
usa_oil_production.set_index('Month', inplace=True)
usa_oil_production.rename_axis('Date',inplace=True)
usa_oil_production['usa_oil_production'] = usa_oil_production['usa_oil_production']*1000 
usa_oil_production.index= usa_oil_production.index.astype('datetime64[ns]')
usa_oil_production

,usa_oil_production
Date,
1920-04-01,1165000
1921-04-01,1341000
1922-04-01,1506000
1923-04-01,1967000
1924-04-01,1994000
...,...
2019-09-01,12584000
2020-09-01,10926000
2021-09-01,10921000


In [10]:
financial_df = get_financial_data(financial_api_keys,start_date,end_date)
financial_df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[            crude_price
 Date                   
 2000-08-23    32.049999
 2000-08-24    31.629999
 2000-08-25    32.049999
 2000-08-28    32.869999
 2000-08-29    32.720001
 ...                 ...
 2024-09-13    68.650002
 2024-09-16    70.089996
 2024-09-17    71.190002
 2024-09-18    70.910004
 2024-09-19    71.949997
 
 [6044 rows x 1 columns],
              sp_500
 Date               
 2000-09-18  1467.50
 2000-09-19  1478.50
 2000-09-20  1469.50
 2000-09-21  1469.50
 2000-09-22  1468.50
 ...             ...
 2024-09-16  5638.00
 2024-09-17  5639.00
 2024-09-18  5620.75
 2024-09-19  5717.75
 2024-09-20  5702.50
 
 [6064 rows x 1 columns],
                   vix
 Date                 
 1990-01-02  17.240000
 1990-01-03  18.190001
 1990-01-04  19.219999
 1990-01-05  20.110001
 1990-01-08  20.260000
 ...               ...
 2024-09-16  17.139999
 2024-09-17  17.610001
 2024-09-18  18.230000
 2024-09-19  16.330000
 2024-09-20  15.930000
 
 [8747 rows x 1 columns],
                   

In [11]:
# Importing oil consumption data 
world_oil_consumption = pd.read_csv("../data/world_oil_consumption.csv")
world_oil_consumption.rename(columns={"Oil consumption - TWh":"world_oil_consumption"},inplace=True)
world_oil_consumption.drop(world_oil_consumption[world_oil_consumption['Entity'] != "World"].index, inplace=True)
world_oil_consumption.drop(columns={'Entity','Code'},inplace=True)